In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from nltk.stem.snowball import SnowballStemmer

print("Done...")

Done...


### Process of Formatting Data:
1. Move all sentences into frequency feature vector, unigram model

2. Move all tags into useable format for additive model accuracy

3. Run data through one of the following models:

    a. Recurrent Neural Network
    
    b. Heirarchical Clustering Method
    
    c. Decision Tree Regressor 
    

### Part 1: Moving Sentences into Binary Feature Vector, UNI & BI models
#### Things made for training include... 

    descriptions_train
    
    features_train (useless)
    
    images_train
    
    tags_train

#### Things made for testing include... 
    
    descriptions_test
    
    features_trest
    
    images_test
    
    tags_test
    

In [2]:
train_descriptions = './all_data/descriptions_train'
files = []
for i in os.listdir(train_descriptions):
    files.append(i)
print(len(files))

10000


In [3]:

def load_data():
    d = []
    for f in files:
        sentences_per_file = []
        for line in open(train_descriptions+'/'+str(f)):
            #stripping punctuation from the line
            line = re.sub(r'[^\w\s]','',line)
            #splitting the line into a list based on spaces
            line = line.split(' ')
            #removing the last two characters of the last item in the list
            line[-1] = line[-1][:-2]
            #appending the cleaned line to our array
            sentences_per_file.append(line)
        d.append(sentences_per_file)

    #appending each chunk of sentences per file to our total raw train data
    return d

raw_train_descriptions = load_data()
#printing length. Length should be equal to our number of files
print(len(raw_train_descriptions))

10000


#### Cleaning the data

In [4]:
#Initialized stemmer object and language
stemmer = SnowballStemmer('english')

#Running list of stop words, can add more...
stop_words = ['a', 'and', 'but', 'to', 'it', 
              'is', 'a', 'the', 'of', 'as', 
             'this', 'that', 'was']

def stem():
    '''
    Returns a cleaned, stemmed list of lists in the same format
    as the raw_train_descriptions. 
    Returns a list of all words seen in the dataset.
    
    Stemmer takes in a word like 'generously' and returns 'generous'
    from online sources, snowball seems to be a favorite!
    '''
    cleaned_words = []
    stemmed_train_descriptions = []
    for chunk in raw_train_descriptions:
        new_chunk = []
        for sentence in chunk:
            new_sentence = []
            for word in sentence:
                word = word.lower()
                if word not in stop_words:
                    clean = stemmer.stem(word)
                    cleaned_words.append(clean)
                    new_sentence.append(clean)
            new_chunk.append(new_sentence)
        stemmed_train_descriptions.append(new_chunk)
    return stemmed_train_descriptions, cleaned_words

stemmed_train, cleaned_words = stem()

print(len(stemmed_train), len(cleaned_words))

(10000, 382838)


In [5]:
#Getting a set() of the cleaned_words for the feature vector
set_cleaned_words = list(set(cleaned_words))
print(len(set_cleaned_words))
print('LOL, only 8000 unique words!')

print(cleaned_words[:10])
print('\n\n') 
print(set_cleaned_words[:10])

7997
LOL, only 8000 unique words!
[u'skateboard', u'put', 'on', u'show', u'use', u'picnic', u'tabl', u'his', u'stag', u'skateboard']



['', u'fourfiv', u'birdfeed', u'childern', u'woodi', u'yellow', u'four', u'sunlit', u'digit', u'deli']


### Making a smaller dataframe where each 5 sentence reviews are combined into 1 list...

In [6]:
def condenser(list_of_lists):
    total = []
    for chunk in list_of_lists:
        chunk_of_words = []
        for sentence in chunk:
            for word in sentence:
                chunk_of_words.append(word)
        total.append(chunk_of_words)
    return total
stemmed_train_per_image = condenser(stemmed_train)
print(len(stemmed_train_per_image))
print(stemmed_train_per_image[:2])

10000
[[u'skateboard', u'put', 'on', u'show', u'use', u'picnic', u'tabl', u'his', u'stag', u'skateboard', u'pull', u'trick', 'on', u'top', u'picnic', u'tabl', u'man', u'ride', 'on', u'skateboard', 'on', u'top', u'tabl', u'skate', u'boarder', u'do', u'trick', 'on', u'picnic', u'tabl', u'person', u'ride', u'skateboard', 'on', u'picnic', u'tabl', u'with', u'crowd', u'watchin'], [u'bowl', u'soup', u'has', u'some', u'carrot', u'shrimp', u'noodl', 'in', 'i', u'healthi', u'food', 'in', u'bowl', u'readi', u'eat', '', u'soup', u'has', u'carrot', u'shrimp', 'in', u'sit', u'next', u'chopstick', u'tasti', u'bowl', u'ramen', u'serv', u'for', u'someon', u'enjoy', '', u'bowl', u'asian', u'noodl', u'soup', u'with', u'shrimp', u'carrot']]


### Its Feature vector time!!! 
##### First creating a unigram vector, then a bigram vector

In [11]:
unigram2 = pd.DataFrame(0, index = range(len(raw_train_descriptions)), \
                      columns = set_cleaned_words)
#runtime ~~ 7 min
def check_data(data, index):
    return data[index]

for col in unigram2.columns:
#     print(col)
    for row in range(len(unigram2)):
        for word in stemmed_train_per_image[row]:
            if col == word:
                unigram2[col][row] += 1
#         for word in stemmed_train_per_image[row]:
#             if word == col:
#                 unigram[col][row] += 1
print(unigram2.head())
unigram2.to_csv('unigram')


      fourfiv  birdfeed  childern  woodi  yellow  four  sunlit  digit  deli  \
0  0        0         0         0      0       0     0       0      0     0   
1  2        0         0         0      0       0     0       0      0     0   
2  2        0         0         0      0       0     0       0      0     0   
3  1        0         0         0      0       0     0       0      0     0   
4  2        0         0         0      0       0     0       0      0     0   

   ...   humbl  cliff  pringl  tableclot  experienc  floret  tomot  pigeo  \
0  ...       0      0       0          0          0       0      0      0   
1  ...       0      0       0          0          0       0      0      0   
2  ...       0      0       0          0          0       0      0      0   
3  ...       0      0       0          0          0       0      0      0   
4  ...       0      0       0          0          0       0      0      0   

   emerg  sash  
0      0     0  
1      0     0  
2      0   

## Helper / Scratch Work (for checking and confirming logic only)

In [8]:
# unigram = pd.DataFrame(0, index = range(len(raw_train_descriptions)), \
#                       columns = set_cleaned_words)
# def check_data(data, index):
#     return data[index]

# for col in unigram.columns:
# #     print(col)
#     for row in range(len(unigram)):
#         if col in stemmed_train_per_image[row]:
#             unigram[col][row] += 1
# #         for word in stemmed_train_per_image[row]:
# #             if word == col:
# #                 unigram[col][row] += 1
# print(unigram.head())

      fourfiv  birdfeed  childern  woodi  yellow  four  sunlit  digit  deli  \
0  0        0         0         0      0       0     0       0      0     0   
1  1        0         0         0      0       0     0       0      0     0   
2  1        0         0         0      0       0     0       0      0     0   
3  1        0         0         0      0       0     0       0      0     0   
4  1        0         0         0      0       0     0       0      0     0   

   ...   humbl  cliff  pringl  tableclot  experienc  floret  tomot  pigeo  \
0  ...       0      0       0          0          0       0      0      0   
1  ...       0      0       0          0          0       0      0      0   
2  ...       0      0       0          0          0       0      0      0   
3  ...       0      0       0          0          0       0      0      0   
4  ...       0      0       0          0          0       0      0      0   

   emerg  sash  
0      0     0  
1      0     0  
2      0   

In [9]:
# print(unigram.shape)
print(unigram2.shape)

(10000, 7997)
(10000, 7997)


In [10]:
#Making sure im not crazy!
# print(unigram.head())
# print(unigram.shape)
print('skateboard' in unigram2.columns)

#Testing the gram model small scale...
cols = ['a', 'b', 'c', 'aa', 'bb', 'cc']

test_data = [[['a', 'a'], ['b', 'b'], ['cc']],
             [['a'], ['b'], ['c']],
             [['aa', 'bb', 'cc'], ['a'], ['b']],
             [['a'], ['b'], ['c', 'cc', 'bb', 'aa']]]

test_df = pd.DataFrame(0, index = range(len(test_data)), columns = cols)
print(test_df)
print(len(test_df))
def check_data(data, index):
    return data[index]
print check_data(test_data, 2)

# for col in test_df.columns:
# #    print col
#     for row in range(test_df.shape[0]):
#         index_list = check_data(test_data, row)
#         for i in index_list:
#             for j in i:
#                 if j == col:
#                     test_df[col][row] += 1
        

print(test_df)

g = 0
for i in unigram['boarder']:
    if i == 1:
        g += 1
print g, 'done'
g = 0
for i in unigram2['boarder']:
    if i != 0:
        g += 1
print g, 'done'

(10000, 7997)
True
   a  b  c  aa  bb  cc
0  0  0  0   0   0   0
1  0  0  0   0   0   0
2  0  0  0   0   0   0
3  0  0  0   0   0   0
4
[['aa', 'bb', 'cc'], ['a'], ['b']]
   a  b  c  aa  bb  cc
0  0  0  0   0   0   0
1  0  0  0   0   0   0
2  0  0  0   0   0   0
3  0  0  0   0   0   0
44 done
44 done
